In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np

In [23]:
df = pd.read_csv('data.csv')

In [24]:
def quit_space(text):
    text = text.lower()
    text = text.replace('\n', ' ')
    text = text.replace('\t', ' ')
    return text

In [25]:
from sklearn.model_selection import train_test_split

df['Code1'] = df['Code1'].apply(quit_space)
df['Code2'] = df['Code2'].apply(quit_space)

x_features = df[['Code1', 'Code2']]
y_labels = df['Plagio']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(x_features, y_labels, test_size=0.2, random_state=42)

# Tokenización y padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train['Code1'] + X_train['Code2'])

X_train_code1_sequences = tokenizer.texts_to_sequences(X_train['Code1'])
X_train_code2_sequences = tokenizer.texts_to_sequences(X_train['Code2'])
X_test_code1_sequences = tokenizer.texts_to_sequences(X_test['Code1'])
X_test_code2_sequences = tokenizer.texts_to_sequences(X_test['Code2'])

max_length = max(max(len(seq) for seq in X_train_code1_sequences), max(len(seq) for seq in X_train_code2_sequences),
                 max(len(seq) for seq in X_train_code1_sequences), max(len(seq) for seq in X_train_code2_sequences))

X_train_code1_padded = pad_sequences(X_train_code1_sequences, maxlen=max_length)
X_train_code2_padded = pad_sequences(X_train_code2_sequences, maxlen=max_length)
X_test_code1_padded = pad_sequences(X_test_code1_sequences, maxlen=max_length)
X_test_code2_padded = pad_sequences(X_test_code2_sequences, maxlen=max_length)

# Concatenar las secuencias de Code1 y Code2
X_train_combined = np.hstack((X_train_code1_padded, X_train_code2_padded))
X_test_combined = np.hstack((X_test_code1_padded, X_test_code2_padded))

print(max_length)


5417


In [26]:
input_dim = max(tokenizer.word_index.values()) + 1
classifier = RandomForestClassifier()
classifier.fit(X_train_code1_padded + X_train_code2_padded, y_train)

RandomForestClassifier()

In [32]:

X_train_code1_padded 

array([[0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1]])

In [27]:
from sklearn.metrics import confusion_matrix

# Predicciones en los datos de prueba
y_pred = classifier.predict([X_test_code1_padded, X_test_code2_padded])
y_pred_classes = np.round(y_pred)

# Crear la matriz de confusión
conf_matrix = confusion_matrix(y_test, y_pred_classes)

print("Matriz de Confusión:")
print(conf_matrix)
print(classification_report(y_test, y_pred))

ValueError: X has 10834 features, but DecisionTreeClassifier is expecting 5417 features as input.

In [ ]:
# Convertir las probabilidades en clases predichas (ajusta según tu problema de clasificación)
y_pred_test = (y_pred > 0.5).astype("int32")  # Para problemas binarios

# Calcular F1 y Recall
f1 = f1_score(y_test, y_pred_test, average="macro")
recall = recall_score(y_test, y_pred_test, average="macro")

print("F1 Score y_pred_test:", f1)
print("Recall y_pred_test:", recall)
print("\n")

# Para obtener un reporte completo
report = classification_report(y_test, y_pred_test)
print("Classification Report:\n", report)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("F1 Score y_pred: {:.2f}%".format(f1 * 100))
print("Recall y_pred: {:.2f}%".format(recall * 100))
print("Accuracy: {:.2f}%".format(accuracy * 100))
